In [1]:
# !pip install pypdf
# !pip install langchain
# !pip install google-cloud-storage
# !pip install openai
# !pip install tiktoken
# !pip install psycopg2
# !pip install cloud-sql-python-connector
# !pip install asyncio asyncpg
# !pip install pgvector
# !pip install PyPDF2
# !pip install pypdf

In [2]:
import os
import openai
import sys
from datetime import datetime, timedelta

from sql_queries import embedding_related_created_queries

from google.cloud.sql.connector import Connector
from google.cloud import storage
import asyncio
import asyncpg
import nest_asyncio
import pickle

from pgvector.asyncpg import register_vector
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings.openai import OpenAIEmbeddings

from langchain.text_splitter import RecursiveCharacterTextSplitter

import pandas as pd

In [3]:
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 

openai.api_key  = os.environ['OPENAI_API_KEY']

## UPDATE DATA info From Cloud Storage to CloudSQL

In [13]:
async def get_blob():

    storage_client = storage.Client.from_service_account_json(str(os.getenv('STORAGE_SERVICE_JSON')))
    
    bucket_name = os.getenv('STORAGE_NAME')
    bucket = storage_client.get_bucket(bucket_name)
    
    blobs = bucket.list_blobs()

loop_get_blob = asyncio.get_event_loop()

# task_get_blob = loop_get_blob.create_task(update_storage_to_cloudsql())

In [14]:
insert_query_storage = """
    INSERT INTO document_storage (document_storage_dir, document_storage_filename, is_document_embedded) VALUES ($1, $2, $3);
"""

async def update_storage_to_cloudsql():
    conn = await asyncpg.connect(
        host=os.getenv('DB_HOST'),
        database=os.getenv('DB_NAME'),
        user=os.getenv('DB_USER'),
        password=os.getenv('DB_PASS'),
        port=os.getenv('DB_PORT')
    )

    count = 0

    blob = loop_get_blob.create_task(update_storage_to_cloudsql())
    
    for blob in blobs:
        
        dir_file_name = blob.name
        downloaded_file_name = dir_file_name.split('/')[-1]
        if downloaded_file_name.endswith('.pdf'):
            existing_dir = await conn.fetchval('SELECT document_storage_dir FROM document_storage WHERE document_storage_dir = $1', dir_file_name)
            if not existing_dir:
                await conn.execute(insert_query_storage, dir_file_name, downloaded_file_name, blob.public_url, False)
                print('Updated new document data')
                count += 1
    if count == 0:
        print("Don't have new document updated from Cloud Storage!")

loop_update_storage_to_cloudsql = asyncio.get_event_loop()

task_update_storage_to_cloudsql = loop_update_storage_to_cloudsql.create_task(update_storage_to_cloudsql())

## CREATE table if NOT EXISTS related to embedding process

In [5]:
async def connect_create(queries):
    conn = await asyncpg.connect(
        host=os.getenv('DB_HOST'),
        database=os.getenv('DB_NAME'),
        user=os.getenv('DB_USER'),
        password=os.getenv('DB_PASS'),
        port=os.getenv('DB_PORT')
    )
    for query in queries:
        await conn.execute(query)
        print("Query created successfully.")

    await conn.close()

loop_connect_create = asyncio.get_event_loop()

task_connect_create = loop_connect_create.create_task(connect_create(embedding_related_created_queries))

In [6]:
async def task_for_counting():
    conn = await asyncpg.connect(
        host=os.getenv('DB_HOST'),
        database=os.getenv('DB_NAME'),
        user=os.getenv('DB_USER'),
        password=os.getenv('DB_PASS'),
        port=os.getenv('DB_PORT')
    )
    
    # list_file_need_embedded
    q_list_file_need_embedded = """
        SELECT document_storage_filename FROM document_storage WHERE is_document_embedded = false;
    """
    
    records = await conn.fetch(q_list_file_need_embedded)
    list_file_need_embedded = [record['document_storage_filename'] for record in records]
    
    # number_of_embedded_file
    q_number_of_embedded_file = """
    SELECT count(document_storage_filename) as number_of_embedded_file FROM document_storage WHERE is_document_embedded = true;
    """    
    records = await conn.fetch(q_number_of_embedded_file)
    number_of_embedded_file = [record['number_of_embedded_file'] for record in records]

    await conn.close()
    
    return list_file_need_embedded, number_of_embedded_file[0]

loop_task_for_counting = asyncio.get_event_loop()
nest_asyncio.apply()
list_file_need_embedded, number_of_embedded_file = loop_task_for_counting.run_until_complete(task_for_counting())

Exception in callback <TaskStepMethWrapper object at 0x115643580>()
handle: <Handle <TaskStepMethWrapper object at 0x115643580>()>
Traceback (most recent call last):
  File "/Users/thailinhpham/anaconda3/envs/t1/lib/python3.11/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: Cannot enter into task <Task pending name='Task-5' coro=<update_storage_to_cloudsql() running at /var/folders/88/l6n2p6zj7k511lqxlc10jcvh0000gn/T/ipykernel_41833/2991273364.py:12>> while another task <Task pending name='Task-3' coro=<Kernel.dispatch_queue() running at /Users/thailinhpham/anaconda3/envs/t1/lib/python3.11/site-packages/ipykernel/kernelbase.py:516> cb=[IOLoop.add_future.<locals>.<lambda>() at /Users/thailinhpham/anaconda3/envs/t1/lib/python3.11/site-packages/tornado/ioloop.py:685]> is being executed.
Exception in callback <TaskStepMethWrapper object at 0x115643d00>()
handle: <Handle <TaskStepMethWrapper object at 0x115643d00>()>
Traceback (most rece

In [10]:
async def get_text_for_embbed():
    count = 0
    q_update_embedded = """
        UPDATE document_storage
        SET is_document_embedded = true
        WHERE document_storage_filename = ANY($1);
    """
    loaders = []
    
    
    storage_client = storage.Client.from_service_account_json(str(os.getenv('STORAGE_SERVICE_JSON')))
    
    bucket_name = os.getenv('STORAGE_NAME')
    bucket = storage_client.get_bucket(bucket_name)
    
    blobs = bucket.list_blobs()
    
    
    for blob in blobs:
        dir_file_name = blob.name
        downloaded_file_name = dir_file_name.split('/')[-1]
        if downloaded_file_name in list_file_need_embedded:
            if downloaded_file_name.endswith('.pdf'):
                count += 1
                blob.download_to_filename(downloaded_file_name)
                # print(f"The file {downloaded_file_name} has been downloaded.")
                loaders.append(PyPDFLoader(downloaded_file_name))
                await conn.execute(q_update_embedded, (downloaded_file_name,))
    
    print(f"{count} files have been downloaded.")
    docs = []
    
    
    
    for loader in loaders:
        docs.extend(loader.load())
    
    text_splitter = RecursiveCharacterTextSplitter(
        separators=[".", "\n"],
        chunk_size = 1536,
        chunk_overlap = 0
    )
    
    texts = text_splitter.split_documents(docs)

    

0 files have been downloaded.


In [11]:
texts

[]

In [12]:
insert_query = """
    INSERT INTO document_embedding (chunk_file, privilege_id, tag, page_content, embedding) VALUES ($1, $2, $3, $4, $5);
"""

async def connect_execute(insert_query, chunk_file, privilege_id, tag,  splits):
    conn = await asyncpg.connect(
        host=os.getenv('DB_HOST'),
        database=os.getenv('DB_NAME'),
        user=os.getenv('DB_USER'),
        password=os.getenv('DB_PASS'),
        port=os.getenv('DB_PORT')
    )

    
    await conn.execute("CREATE EXTENSION IF NOT EXISTS vector")
    await register_vector(conn)

    embeddings = OpenAIEmbeddings()
    doc_vector = embeddings.embed_documents([t.page_content for t in texts])

    q_update_embedded = """
        UPDATE document_storage
        SET is_document_embedded = true
        WHERE document_storage_filename = ANY($1);
    """
for file in list_file_need_embedded:
    await conn.execute(q_update_embedded, (file,))

    i = -1 
    count_success = 0
    for split in splits:
        try:
            i = i + 1
            chunk_file +=1
            await conn.execute(insert_query, chunk_file, privilege_id, tag, split.page_content, doc_vector[i])
            count_success += 1
        except Exception as e:
            print(f"Failed to add split: {e}")

    await conn.close()

    percent = count_success / (i + 1) * 100
    print(f'Embedded with {percent}% new files')


loop = asyncio.get_event_loop()
# number_of_embedded_file (chunk_file) will be continue count up as each embbed file has been vectorized

tag = 'CV'
privilege_id = 4

embbed = loop.create_task(connect_execute(insert_query, number_of_embedded_file, privilege_id, tag, texts))

In [29]:
class StorageManager:
    def __init__(self, service_account_json, bucket_name):
        self.client = storage.Client.from_service_account_json(service_account_json)
        self.bucket = self.client.bucket(bucket_name)

    async def fetch_pdf_filenames(self):
        blobs = [blob for blob in self.bucket.list_blobs() if blob.name.lower().endswith('.pdf')]
        return [(blob.name, os.path.basename(blob.name)) for blob in blobs]

class DatabaseManager:
    def __init__(self, db_credentials):
        self.db_credentials = db_credentials
        self.pool = None

    async def create_pool(self):
        if not self.pool:
            self.pool = await asyncpg.create_pool(**self.db_credentials)

    async def close_pool(self):
        if self.pool:
            await self.pool.close()
            self.pool = None

    async def create_tables(self):
        await self.create_pool()
        async with self.pool.acquire() as conn:
            for query in embedding_related_created_queries:
                await conn.execute(query)

    async def document_exists(self, filename):
        async with self.pool.acquire() as conn:
            return await conn.fetchval(
                "SELECT EXISTS(SELECT 1 FROM document_storage WHERE document_storage_filename = $1)",
                filename
            )

    async def insert_document_storage(self, dir_name, filename, is_embedded):
        async with self.pool.acquire() as conn:
            if not await self.document_exists(filename):
                await conn.execute(
                    "INSERT INTO document_storage (document_storage_dir, document_storage_filename, is_document_embedded) "
                    "VALUES ($1, $2, $3)",
                    dir_name, filename, is_embedded
                )

    async def fetch_documents_to_embed(self):
        async with self.pool.acquire() as conn:
            return await conn.fetch(
                "SELECT document_storage_filename FROM document_storage WHERE is_document_embedded = false"
            )

    async def insert_document_embedding(self, chunk_file, privilege_id, tag, page_content, embedding):
        async with self.pool.acquire() as conn:
            await conn.execute(
                "INSERT INTO document_embedding (chunk_file, privilege_id, tag, page_content, embedding) "
                "VALUES ($1, $2, $3, $4, $5)",
                chunk_file, privilege_id, tag, page_content, embedding
            )

    async def set_document_embedded(self, filename):
        async with self.pool.acquire() as conn:
            await conn.execute(
                "UPDATE document_storage SET is_document_embedded = true WHERE document_storage_filename = $1",
                filename
            )


class DocumentManager:
    def __init__(self, storage_manager, database_manager, openai_api_key):
        self.storage_manager = storage_manager
        self.database_manager = database_manager
        self.openai_api_key = openai_api_key

    async def process_and_embed_documents(self):
        pdf_files = self.storage_manager.fetch_pdf_filenames()
        await self.database_manager.create_pool()  # Make sure the pool is created
        files_to_embed = [record['document_storage_filename'] for record in
                          await self.database_manager.fetch_documents_to_embed()]

        embeddings = OpenAIEmbeddings(api_key=self.openai_api_key)
        text_splitter = RecursiveCharacterTextSplitter(separators=[".", "\n"], chunk_size=1536, chunk_overlap=0)

        # Process the documents
        for file_path, file_name in pdf_files:
            if file_name in files_to_embed:
                blob = self.storage_manager.bucket.blob(file_path)
                local_file_path = f"/tmp/{file_name}"
                blob.download_to_filename(local_file_path)

                # Load the document content
                loader = PyPDFLoader(local_file_path)
                document = loader.load()

                # Split and embed the text
                splits = text_splitter.split_documents([document])
                doc_vector = embeddings.embed_documents([split.page_content for split in splits])

                for i, split in enumerate(splits):
                    # Insert embedding into database
                    await self.database_manager.insert_document_embedding(2, 4, 'CV', split.page_content,
                                                                         doc_vector[i])

                # Update the document_storage as embedded
                await self.database_manager.set_document_embedded(file_name)

async def main():
    db_manager = DatabaseManager({
        'host': os.getenv('DB_HOST'),
        'database': os.getenv('DB_NAME'),
        'user': os.getenv('DB_USER'),
        'password': os.getenv('DB_PASS'),
        'port': os.getenv('DB_PORT')
    })
    # # Initialize the pool and create tables
    storage_manager = StorageManager(os.getenv('STORAGE_SERVICE_JSON'), os.getenv('STORAGE_NAME'))
    document_manager = DocumentManager(storage_manager, db_manager, os.getenv('OPENAI_API_KEY'))
    # # Process documents
    await document_manager.process_and_embed_documents()

if __name__ == "__main__":
    asyncio.run(main())

TypeError: 'coroutine' object is not iterable

In [42]:
class StorageManager:
    def __init__(self, service_account_json, bucket_name):
        self.client = storage.Client.from_service_account_json(service_account_json)
        self.bucket = self.client.bucket(bucket_name)

    def fetch_pdf_filenames(self):
        return [(blob.name, os.path.basename(blob.name)) for blob in self.bucket.list_blobs() if blob.name.lower().endswith('.pdf')]

class DatabaseManager:
    def __init__(self, db_credentials):
        self.db_credentials = db_credentials
        self.pool = None

    async def create_pool(self):
        if not self.pool:
            self.pool = await asyncpg.create_pool(**self.db_credentials)

    async def close_pool(self):
        if self.pool:
            await self.pool.close()
            self.pool = None

    async def create_tables(self):
        await self.create_pool()
        async with self.pool.acquire() as conn:
            for query in embedding_related_created_queries:
                await conn.execute(query)

    async def document_exists(self, filename):
        async with self.pool.acquire() as conn:
            return await conn.fetchval(
                "SELECT EXISTS(SELECT 1 FROM document_storage WHERE document_storage_filename = $1)",
                filename
            )

    async def insert_document_storage(self, dir_name, filename, is_embedded):
        async with self.pool.acquire() as conn:
            if not await self.document_exists(filename):
                await conn.execute(
                    "INSERT INTO document_storage (document_storage_dir, document_storage_filename, is_document_embedded) "
                    "VALUES ($1, $2, $3)",
                    dir_name, filename, is_embedded
                )

    async def fetch_documents_to_embed(self):
        async with self.pool.acquire() as conn:
            return await conn.fetch(
                "SELECT document_storage_filename FROM document_storage WHERE is_document_embedded = false"
            )

    async def insert_document_embedding(self, chunk_file, privilege_id, tag, page_content, embedding):
        async with self.pool.acquire() as conn:
            await conn.execute(
                "INSERT INTO document_embedding (chunk_file, privilege_id, tag, page_content, embedding) "
                "VALUES ($1, $2, $3, $4, $5)",
                chunk_file, privilege_id, tag, page_content, embedding
            )

    async def set_document_embedded(self, filename):
        async with self.pool.acquire() as conn:
            await conn.execute(
                "UPDATE document_storage SET is_document_embedded = true WHERE document_storage_filename = $1",
                filename
            )

class DocumentManager:
    def __init__(self, storage_manager, database_manager, openai_api_key):
        self.storage_manager = storage_manager
        self.database_manager = database_manager
        self.openai_api_key = openai_api_key

    async def process_and_embed_documents(self):
        pdf_files = self.storage_manager.fetch_pdf_filenames()
        await self.database_manager.create_pool()  # Ensure the connection pool is created
        files_to_embed = [filename for _, filename in pdf_files]
        files_to_embed_records = await self.database_manager.fetch_documents_to_embed()

        # Filter the files based on whether they need to be embedded.
        files_to_embed = {record['document_storage_filename'] for record in files_to_embed_records}.intersection(files_to_embed)

        embeddings = OpenAIEmbeddings(api_key=self.openai_api_key)
        text_splitter = RecursiveCharacterTextSplitter(separators=[".", "\n"], chunk_size=1536, chunk_overlap=0)

        # Process the documents
        for file_path, file_name in pdf_files:
            if file_name in files_to_embed:
                # print(f'{file_name} file needs to be embedd')
                blob = self.storage_manager.bucket.blob(file_path)
                print(file_path)
                # local_file_path = f"{}"
                print(local_file_path)
                blob.download_to_filename(local_file_path)

                # Load the document content
                loader = PyPDFLoader(local_file_path)
                document = loader.load()

                # Split and embed the text
                splits = text_splitter.split_documents(document)
                doc_vectors = embeddings.embed_documents([t.page_content for t in texts]) 

                i = -1
                
                for split in splits:
                    # Insert embedding into the database
                    print(doc_vectors)
                    i += 1
                    await self.database_manager.insert_document_embedding(i, 4, 'CV', split.page_content, doc_vectors[i])  # Adjust the i+1 based on your chunk_file id strategy
                    
                # Update the document_storage as embedded
                await self.database_manager.set_document_embedded(file_name)

        # After processing, close the pool
        await self.database_manager.close_pool()
        print('Done')

async def main():
    db_manager = DatabaseManager({
        'host': os.getenv('DB_HOST'),
        'database': os.getenv('DB_NAME'),
        'user': os.getenv('DB_USER'),
        'password': os.getenv('DB_PASS'),
        'port': os.getenv('DB_PORT')
    })
    # # Initialize the pool and create tables
    storage_manager = StorageManager(os.getenv('STORAGE_SERVICE_JSON'), os.getenv('STORAGE_NAME'))
    document_manager = DocumentManager(storage_manager, db_manager, os.getenv('OPENAI_API_KEY'))
    # # Process documents
    await document_manager.process_and_embed_documents()

if __name__ == "__main__":
    asyncio.run(main())

TooManyConnectionsError: sorry, too many clients already :: proc.c:360